Constraint combination exercise
You are working on a distribution plan for a warehouse network. The network has two warehouses (W1, and W2) and each can ship three different types of products (A, B, and C). W1 is small and can either ship 10 products A per a week or 15 products B per a week or 20 products C per a week. You are looking to minimize the total costs.

A Pandas DataFrame named demand is printed in the console and contains the monthly demand for each product. Additionally, the code of the PuLP model to initialize, define decision variables, objective function, and constraint so total shipment of each product equals its demand.


Complete the code for the constraint that models the shipping limitations of warehouse W1 for 4 weeks.

In [1]:
import pandas as pd
from pulp import *

In [2]:
d = {'C1':[10,17,23], 'C2':[8,11,20], 'C3':[28,10,25], 'C4':[0,6,13], 'Product':['A','B','C']}
c = {'C1':[81,117,107,102,189,108], 'C2':[92,77,97,105,132,98], 'C3':[110,108,102,115,200,103], 'C4':[130,98,104,130,192,105],
    'Product':['A','B','C','A','B','C'], 'WH':['W1','W1','W1','W2','W2','W2']}

demand = pd.DataFrame(data = d)
demand = demand.set_index('Product')

costs = pd.DataFrame(data = c)
costs = costs.set_index(['WH','Product'])

In [3]:
print(demand)
print(costs)

         C1  C2  C3  C4
Product                
A        10   8  28   0
B        17  11  10   6
C        23  20  25  13
             C1   C2   C3   C4
WH Product                    
W1 A         81   92  110  130
   B        117   77  108   98
   C        107   97  102  104
W2 A        102  105  115  130
   B        189  132  200  192
   C        108   98  103  105


In [4]:
# Initialize, Define Decision Vars., Objective Function, and Constraints
model = LpProblem("Distribution Planning", LpMinimize)
wh = ['W1','W2']
prod = ['A', 'B', 'C']
cust = ['C1', 'C2', 'C3', 'C4']

In [5]:
X = LpVariable.dicts("ship", [(w, p, c) for c in cust for p in prod for w in wh], 
                     lowBound=0, cat="Integer")


In [6]:
print(X)

{('W1', 'A', 'C1'): ship_('W1',_'A',_'C1'), ('W2', 'A', 'C1'): ship_('W2',_'A',_'C1'), ('W1', 'B', 'C1'): ship_('W1',_'B',_'C1'), ('W2', 'B', 'C1'): ship_('W2',_'B',_'C1'), ('W1', 'C', 'C1'): ship_('W1',_'C',_'C1'), ('W2', 'C', 'C1'): ship_('W2',_'C',_'C1'), ('W1', 'A', 'C2'): ship_('W1',_'A',_'C2'), ('W2', 'A', 'C2'): ship_('W2',_'A',_'C2'), ('W1', 'B', 'C2'): ship_('W1',_'B',_'C2'), ('W2', 'B', 'C2'): ship_('W2',_'B',_'C2'), ('W1', 'C', 'C2'): ship_('W1',_'C',_'C2'), ('W2', 'C', 'C2'): ship_('W2',_'C',_'C2'), ('W1', 'A', 'C3'): ship_('W1',_'A',_'C3'), ('W2', 'A', 'C3'): ship_('W2',_'A',_'C3'), ('W1', 'B', 'C3'): ship_('W1',_'B',_'C3'), ('W2', 'B', 'C3'): ship_('W2',_'B',_'C3'), ('W1', 'C', 'C3'): ship_('W1',_'C',_'C3'), ('W2', 'C', 'C3'): ship_('W2',_'C',_'C3'), ('W1', 'A', 'C4'): ship_('W1',_'A',_'C4'), ('W2', 'A', 'C4'): ship_('W2',_'A',_'C4'), ('W1', 'B', 'C4'): ship_('W1',_'B',_'C4'), ('W2', 'B', 'C4'): ship_('W2',_'B',_'C4'), ('W1', 'C', 'C4'): ship_('W1',_'C',_'C4'), ('W2', 'C'

In [7]:
for c in cust:
    for p in prod:
        for w in wh:
            print(costs.loc[(w, p), c])

81
102
117
189
107
108
92
105
77
132
97
98
110
115
108
200
102
103
130
130
98
192
104
105


In [8]:
model += lpSum([ X[(w, p, c)] * costs.loc[(w, p), c] for c in cust for p in prod for w in wh ])

In [9]:
for c in cust:
    for p in prod:
        model += lpSum([X[(w, p, c)] for w in wh]) == demand.loc[p, c]

In [10]:
# Define Dependent Demand Constraints
model += ((1/10) * lpSum([X[('W1', 'A', c)] for c in cust]) 
          + (1/15) * lpSum([X[('W1', 'B', c)] for c in cust])
          + (1/20) * lpSum([X[('W1', 'C', c)] for c in cust])) <= 4

In [11]:
model

Distribution Planning:
MINIMIZE
81*ship_('W1',_'A',_'C1') + 92*ship_('W1',_'A',_'C2') + 110*ship_('W1',_'A',_'C3') + 130*ship_('W1',_'A',_'C4') + 117*ship_('W1',_'B',_'C1') + 77*ship_('W1',_'B',_'C2') + 108*ship_('W1',_'B',_'C3') + 98*ship_('W1',_'B',_'C4') + 107*ship_('W1',_'C',_'C1') + 97*ship_('W1',_'C',_'C2') + 102*ship_('W1',_'C',_'C3') + 104*ship_('W1',_'C',_'C4') + 102*ship_('W2',_'A',_'C1') + 105*ship_('W2',_'A',_'C2') + 115*ship_('W2',_'A',_'C3') + 130*ship_('W2',_'A',_'C4') + 189*ship_('W2',_'B',_'C1') + 132*ship_('W2',_'B',_'C2') + 200*ship_('W2',_'B',_'C3') + 192*ship_('W2',_'B',_'C4') + 108*ship_('W2',_'C',_'C1') + 98*ship_('W2',_'C',_'C2') + 103*ship_('W2',_'C',_'C3') + 105*ship_('W2',_'C',_'C4') + 0
SUBJECT TO
_C1: ship_('W1',_'A',_'C1') + ship_('W2',_'A',_'C1') = 10

_C2: ship_('W1',_'B',_'C1') + ship_('W2',_'B',_'C1') = 17

_C3: ship_('W1',_'C',_'C1') + ship_('W2',_'C',_'C1') = 23

_C4: ship_('W1',_'A',_'C2') + ship_('W2',_'A',_'C2') = 8

_C5: ship_('W1',_'B',_'C2') + 

In [12]:
model.solve()
for v in model.variables():
    print(v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen
print("The optimised objective function= ", value(model.objective))

ship_('W1',_'A',_'C1') = 10.0
ship_('W1',_'A',_'C2') = 0.0
ship_('W1',_'A',_'C3') = 0.0
ship_('W1',_'A',_'C4') = 0.0
ship_('W1',_'B',_'C1') = 17.0
ship_('W1',_'B',_'C2') = 11.0
ship_('W1',_'B',_'C3') = 10.0
ship_('W1',_'B',_'C4') = 6.0
ship_('W1',_'C',_'C1') = 0.0
ship_('W1',_'C',_'C2') = 0.0
ship_('W1',_'C',_'C3') = 0.0
ship_('W1',_'C',_'C4') = 1.0
ship_('W2',_'A',_'C1') = 0.0
ship_('W2',_'A',_'C2') = 8.0
ship_('W2',_'A',_'C3') = 28.0
ship_('W2',_'A',_'C4') = 0.0
ship_('W2',_'B',_'C1') = 0.0
ship_('W2',_'B',_'C2') = 0.0
ship_('W2',_'B',_'C3') = 0.0
ship_('W2',_'B',_'C4') = 0.0
ship_('W2',_'C',_'C1') = 23.0
ship_('W2',_'C',_'C2') = 20.0
ship_('W2',_'C',_'C3') = 25.0
ship_('W2',_'C',_'C4') = 12.0
The optimised objective function=  17757.0
